# Autogluon to test various ML models

In [34]:
import sys
import logging
import datetime

nblog = open("./logs/tree_ml_"+str(datetime.datetime.now()).split(' ')[0]+".log", "a+")
sys.stdout.echo = nblog
sys.stderr.echo = nblog

get_ipython().log.handlers[0].stream = nblog
get_ipython().log.setLevel(logging.INFO)

%autosave 5

Autosaving every 5 seconds


In [5]:
# %%time
# data_root = 'https://autogluon.s3.amazonaws.com/datasets/Inc/'
# train_data = TabularDataset(data_root + 'train.csv')
# test_data = TabularDataset(data_root + 'test.csv')

# predictor = TabularPredictor(label='class').fit(train_data=train_data, 
#                                                 excluded_model_types=['CatBoostModel']
#                                                )
# predictions = predictor.predict(test_data)

In [6]:
# predictor.evaluate(test_data, silent=True)

In [46]:
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# from sklearn.datasets import load_diabetes
# from sklearn.inspection import PartialDependenceDisplay
# from sklearn.neural_network import MLPRegressor
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
# from sklearn.tree import DecisionTreeRegressor

pd.set_option("display.max_colwidth", 0)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
import warnings
warnings.filterwarnings("ignore")

# Load data

In [47]:
%%time
df_localness = pd.read_csv('./food_localness_final_oct.csv')
print(df_localness.shape[0])

2887183
CPU times: user 34.4 s, sys: 13.6 s, total: 47.9 s
Wall time: 49.1 s


In [48]:
df_localness.head(2)

,account_id,order_id,order_date,order_time,vendor_id,vendor_name,main_cuisine,vertical,delivery_fee_amount_lc,service_fee_amount_lc,basket_amount_lc,is_incentive_applied,incentive_amount_lc,pickup_id,customer_lon,customer_lat,vendor_lon,vendor_lat,order_time_hour,delivery_duration,total_duration,customer_geometry,vendor_geometry,order_distance_km,n_vertical,n_vendor_id,n_main_cuisine,n_vendor_name,n_vendor_lon,n_vendor_lat,nearest_euc_distance,localness,customer_loc_id,entropy_3.6km,item_price,compensation,discounted delivery,free delivery,gem,item discount,menu discount,voucher wallet,appoxi_rent,entropy_2km,entropy_0.3km,isweekend,poi_count_1km,poi_count_2km,poi_count_3km,poi_count_4km,poi_count_5km,incentives_type,poi_count_1_2km,poi_count_2_3km,poi_count_3_4km,poi_count_4_5km
0,4285647,1305786978,2023-10-10,2023-10-10 18:37:22,15921,"312 , Al Sufouh 1",american,food,9.0,2.36,78.5,0,0.0,249215302.0,55.22939,25.164577,55.175844,25.107854,18,37,54,POINT (6148107.571623116 2895972.7784882844),POINT (6142146.858169098 2888997.883505384),9.174926,food,702934,american,"My Hot Dog, Jumeirah 3",55.221344,25.176179,1.684822,0.183633,55.2293925.164577,0.670995,26.679487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.209394,0.625864,0.000000,False,8.0,96.0,243.0,352.0,570.0,none,88.0,147.0,109.0,218.0
1,1349127,1305612765,2023-10-10,2023-10-10 16:02:54,702587,"Subz, Business Bay",american,food,5.0,2.34,78.0,True,30.0,249170878.0,55.27096,25.180780,55.269598,25.188222,16,10,32,POINT (6152735.122855392 2897965.761873224),POINT (6152583.494576982 2898881.161766449),0.927873,food,702037,american,"Burger Basics, Business Bay",55.275506,25.180872,0.506177,0.545524,55.2709625.18078,0.744623,37.650000,0.0,0.0,0.0,0.0,0.0,30.0,0.0,95.263292,0.753064,0.414145,False,247.0,491.0,798.0,1064.0,1297.0,menu discount,244.0,307.0,266.0,233.0


In [49]:
# df_features = df_localness[['basket_amount_lc','item_price','order_time_hour','total_duration',
#                             'appoxi_rent', 'entropy_3.6km','entropy_2km','entropy_0.3km',
#                             'incentive_amount_lc']]
## eleven features
df_features = df_localness[['basket_amount_lc','item_price','order_time_hour','total_duration',
                            'appoxi_rent', 'entropy_3.6km','entropy_2km','entropy_0.3km',
                            'incentive_amount_lc','isweekend','main_cuisine','incentives_type',
                            'poi_count_1_2km','poi_count_2_3km','poi_count_3_4km','poi_count_4_5km']]

# Check VIF

In [50]:
%%time
df_cor = df_features.corr()
pd.DataFrame(np.linalg.inv(df_features.corr().values), index = df_cor.index, columns=df_cor.columns)

,basket_amount_lc,item_price,order_time_hour,total_duration,appoxi_rent,entropy_3.6km,entropy_2km,entropy_0.3km,incentive_amount_lc,isweekend,poi_count_1_2km,poi_count_2_3km,poi_count_3_4km,poi_count_4_5km
basket_amount_lc,1.130488,-0.173270,-0.046905,-0.223694,-0.066159,-0.025644,0.020493,0.061753,-0.160057,-0.058161,0.010800,0.014018,0.020504,0.005525
item_price,-0.173270,1.080950,-0.021617,-0.014747,-0.063501,-0.031486,-0.003934,-0.005506,-0.180180,0.017358,-0.000882,0.005683,0.012174,-0.005716
order_time_hour,-0.046905,-0.021617,1.014190,-0.070709,0.043952,-0.028117,0.000002,0.023631,-0.003207,0.020871,0.012994,0.014478,0.023656,0.015218
total_duration,-0.223694,-0.014747,-0.070709,1.089934,-0.095593,0.116362,0.023890,-0.024554,-0.078771,0.045789,-0.018028,-0.042774,-0.041390,-0.025224
appoxi_rent,-0.066159,-0.063501,0.043952,-0.095593,1.349343,-0.192220,0.205882,-0.127578,-0.000253,0.017997,-0.467899,-0.149865,0.009478,-0.216783
entropy_3.6km,-0.025644,-0.031486,-0.028117,0.116362,-0.192220,2.794233,-1.969139,0.103287,0.002144,-0.003291,0.041635,-0.202921,-0.226332,-0.224717
entropy_2km,0.020493,-0.003934,0.000002,0.023890,0.205882,-1.969139,3.301752,-0.435561,-0.012157,0.000906,-0.783332,-0.187730,-0.183699,0.023640
entropy_0.3km,0.061753,-0.005506,0.023631,-0.024554,-0.127578,0.103287,-0.435561,1.470070,-0.055314,0.012581,-0.456782,0.028803,-0.172187,0.074262
incentive_amount_lc,-0.160057,-0.180180,-0.003207,-0.078771,-0.000253,0.002144,-0.012157,-0.055314,1.081722,0.012927,-0.006457,0.010485,-0.004290,0.004279
isweekend,-0.058161,0.017358,0.020871,0.045789,0.017997,-0.003291,0.000906,0.012581,0.012927,1.007073,0.010976,0.008468,0.012654,0.007664


In [30]:
X = df_features
y = df_localness.localness

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 123)

# AutoGluon - select the best ML model

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [32]:
train_data = X_train.copy()
train_data['localness'] = y_train
test_data = X_test.copy()
test_data['localness'] = y_test

In [ ]:
%%time
predictor = TabularPredictor(label='localness', problem_type='regression').fit(train_data=train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20240528_133919/"
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20240528_133919/"
AutoGluon Version:  0.6.2
Python Version:     3.7.1
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #167~18.04.1-Ubuntu SMP Wed May 24 00:51:42 UTC 2023
Train Data Rows:    2021028
Train Data Columns: 11
Label Column: localness
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    39879.91 MB
	Train Data (Original)  Memory Usage: 277.4 MB (0.7% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtyp

[1000]	valid_set's rmse: 0.23125
[2000]	valid_set's rmse: 0.225763
[3000]	valid_set's rmse: 0.222001
[4000]	valid_set's rmse: 0.219102
[5000]	valid_set's rmse: 0.216761
[6000]	valid_set's rmse: 0.21477
[7000]	valid_set's rmse: 0.212922
[8000]	valid_set's rmse: 0.211405
[9000]	valid_set's rmse: 0.209956
[10000]	valid_set's rmse: 0.208635


	-0.2086	 = Validation score   (-root_mean_squared_error)
	116.73s	 = Training   runtime
	1.93s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's rmse: 0.220857
[2000]	valid_set's rmse: 0.211793
[3000]	valid_set's rmse: 0.205767
[4000]	valid_set's rmse: 0.201296
[5000]	valid_set's rmse: 0.197656
[6000]	valid_set's rmse: 0.194646
[7000]	valid_set's rmse: 0.19166
[8000]	valid_set's rmse: 0.189324
[9000]	valid_set's rmse: 0.187148
[10000]	valid_set's rmse: 0.18514


	-0.1851	 = Validation score   (-root_mean_squared_error)
	96.6s	 = Training   runtime
	1.65s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-0.2085	 = Validation score   (-root_mean_squared_error)
	289.52s	 = Training   runtime
	0.21s	 = Validation runtime
Fitting model: CatBoost ...


In [ ]:
# Takes a long time, optional step
# predictions = predictor.predict(test_data)

In [45]:
# fittedmodels_eval = predictor.leaderboard(test_data, silent=True)

In [44]:
%%time
# ~7miin
print(datetime.datetime.now(),'************** Running predictor.leaderboard **************')
fittedmodels_eval = predictor.leaderboard(test_data, silent=True)
print(fittedmodels_eval)
print(datetime.datetime.now(),'************** Done predictor.leaderboard **************')

2024-05-28 13:11:09.480429 ************** Running predictor.leaderboard **************
                  model  score_test  score_val  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2 -0.172329   -0.170578   174.923711      3.729644       397.034951   0.053623                 0.025872                0.411438           2            True       12       
1   LightGBMLarge       -0.173432   -0.171644   77.669712       1.608236       131.656050   77.669712                1.608236                131.656050         1            True       11       
2   XGBoost             -0.176700   -0.174880   97.200377       2.095536       264.967464   97.200377                2.095536                264.967464         1            True       9        
3   LightGBM            -0.187838   -0.185139   77.264667       1.645247       96.600144    77.264667                1.645247            

In [43]:
%%time
print(predictor.feature_importance(test_data))

Computing feature importance via permutation shuffling for 11 features using 5000 rows with 5 shuffle sets...
	166.74s	= Expected runtime (33.35s per shuffle set)
	61.47s	= Actual runtime (Completed 5 of 5 shuffle sets)


                     importance    stddev       p_value  n  p99_high   p99_low
main_cuisine         0.124648    0.003104  4.610458e-08  5  0.131039  0.118256
item_price           0.065487    0.003363  8.319024e-07  5  0.072412  0.058562
entropy_2km          0.065389    0.002920  4.757404e-07  5  0.071401  0.059378
entropy_3.6km        0.057811    0.002687  5.581346e-07  5  0.063343  0.052279
total_duration       0.045161    0.002361  8.935943e-07  5  0.050022  0.040299
appoxi_rent          0.038733    0.001097  7.714349e-08  5  0.040992  0.036474
entropy_0.3km        0.028394    0.001016  1.963172e-07  5  0.030485  0.026302
incentive_amount_lc  0.006933    0.000362  8.843926e-07  5  0.007677  0.006188
order_time_hour      0.003574    0.000482  3.866939e-05  5  0.004566  0.002582
basket_amount_lc     0.003505    0.000694  1.747834e-04  5  0.004933  0.002077
isweekend            0.000285    0.000065  3.127074e-04  5  0.000419  0.000150
CPU times: user 23min 25s, sys: 687 ms, total: 23min

In [22]:
# %%time
# predictor.feature_importance(test_data)
# %%time
# predictor.fit_summary()

Computing feature importance via permutation shuffling for 9 features using 5000 rows with 5 shuffle sets...
	64.92s	= Expected runtime (12.98s per shuffle set)
	46.29s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
item_price,0.075773,0.002588,1.631198e-07,5,0.081103,0.070444
entropy_3.6km,0.059856,0.001937,1.314952e-07,5,0.063844,0.055867
entropy_2km,0.059073,0.002024,1.650682e-07,5,0.063240,0.054906
total_duration,0.037830,0.002580,2.578285e-06,5,0.043141,0.032518
appoxi_rent,0.031360,0.001237,2.903121e-07,5,0.033908,0.028812
entropy_0.3km,0.021959,0.001307,1.498310e-06,5,0.024650,0.019269
incentive_amount_lc,0.016241,0.001453,7.616836e-06,5,0.019234,0.013248
basket_amount_lc,0.006863,0.000645,9.242208e-06,5,0.008191,0.005535
order_time_hour,0.004858,0.000391,5.015656e-06,5,0.005664,0.004052
